In [1]:
!pip3 install moviepy torch torchaudio pandas

  Using cached moviepy-1.0.3.tar.gz (388 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/3f/14/e105b8ef6d324e789c1589e95cb0ab63f3e07c2216d68b1178b7c21b7d2a/torch-2.2.2-cp311-none-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/57/c4/80cc3315dd1ca706643b78f894901d4d888ffe376a5e401f73d9db61071e/torchaudio-2.2.2-cp311-cp311-macosx_10_13_x86_64.whl.metadata
  Obtaining dependency information for decorator<5.0,>=4.0.2 from https://files.pythonhosted.org/packages/ed/1b/72a1821152d07cf1d8b6fce298aeb06a7eb90f4d6d41acec9861e7cc6df0/decorator-4.4.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tqdm<5.0,>=4.11.2 from https://files.pythonhosted.org/packages/2a/14/e75e52d521442e2fcc9f1df3c5e456aead034203d4797867980de558a

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 24.0 MB/s eta 0:00:0000:0100:01
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110721 sha256=545a3fe79bfb651919a90bc3882b824a560acaa54902ea0606ef0cd8af40ee55
  Stored in directory: /Users/roshan/Library/Caches/pip/wheels/83/b1/d9/119ef7c144b44d591ec0a9a140465133c23ea95d2a161184ba
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dep

In [2]:
import moviepy.editor as mp
import torch
import torchaudio
import pandas as pd
from tqdm import tqdm

# Define the charset and other constants
CHARSET = " abcdefghijklmnopqrstuvwxyz,.'"
mel_transform = {}

def extract_audio(video_path):
    try:
        video = mp.VideoFileClip(video_path)
        audio_path = "temp_audio.wav"
        video.audio.write_audiofile(audio_path)
        return audio_path
    except Exception as e:
        print(f"Error extracting audio from video: {e}")
        return None

def load_audio(audio_path):
    try:
        waveform, sample_rate = torchaudio.load(audio_path, normalize=True)
        hop_length = int(sample_rate / (1000 / 10))  # 10ms
        win_length = int(sample_rate / (1000 / 25))  # 25ms
        mel_spectrogram = torchaudio.transforms.MelSpectrogram(
            sample_rate, n_fft=win_length, win_length=win_length, hop_length=hop_length, n_mels=80
        )(waveform)
        return mel_spectrogram[0].T  # Transpose to get the correct dimension
    except Exception as e:
        print(f"Error loading audio file: {e}")
        return None

def load_model(path):
    try:
        # Load the model safely
        model = torch.load(path)
        model.eval()  # Set the model to inference mode
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

def recognize_speech(mel_spectrogram, model):
    try:
        if mel_spectrogram is None or model is None:
            raise ValueError("Invalid input or model not loaded")
        
        # Ensure input tensor is correctly shaped
        if len(mel_spectrogram.shape) != 2 or mel_spectrogram.shape[1] != 80:
            raise ValueError("Input mel_spectrogram must have shape [Time, 80]")
        input_tensor = mel_spectrogram.unsqueeze(0)  # Add batch dimension
        output = model(input_tensor)
        predicted_indices = output.argmax(2).squeeze(0)  # Simulate prediction
        text = ''.join([CHARSET[i] for i in predicted_indices if i != 0])
        return text
    except Exception as e:
        print(f"Error during model inference: {e}")
        return ""

def create_csv(data, filename):
    try:
        df = pd.DataFrame(data, columns=["Text", "Video Link"])
        df.to_csv(filename, index=False)
    except Exception as e:
        print(f"Error creating CSV file: {e}")

def video_to_text(video_path, video_link, model_path):
    model = load_model(model_path)
    if model is None:
        print("Model loading failed.")
        return

    audio_path = extract_audio(video_path)
    if audio_path:
        mel_spectrogram = load_audio(audio_path)
        if mel_spectrogram is not None:
            text = recognize_speech(mel_spectrogram, model)
            print(text)
            create_csv([(text, video_link)], "output.csv")
        else:
            print("Failed to process audio for speech recognition.")
    else:
        print("Failed to extract audio from video.")

video_path = "test.mp4"
video_link = "https://www.youtube.com/watch?v=Hb3v7zcu6UY&ab_channel=thenewboston"
model_path = "voice.pt"
video_to_text(video_path, video_link, model_path)

ModuleNotFoundError: No module named 'torchaudio'

In [1]:
!pip3 install torchaudio


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
